In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist

In [2]:
(xtrain, ytrain), (xtest, ytest) = mnist.load_data()

In [3]:
print(xtrain.shape, xtest.shape)
print(ytrain.shape, ytest.shape)

(60000, 28, 28) (10000, 28, 28)
(60000,) (10000,)


In [4]:
# data for pre-trained model
pt_xtrain = []
pt_ytrain = []
pt_xtest = []
pt_ytest = []

# data for transfer learning
tl_xtrain = []
tl_ytrain = []
tl_xtest = []
tl_ytest = []

m = 20000
for i in range(m):
    if ytrain[i]<5:
        pt_xtrain.append(xtrain[i] / 255)
        pt_ytrain.append(ytrain[i])
    else:
        tl_xtrain.append(xtrain[i] / 255)
        tl_ytrain.append(ytrain[i])

m2 = 10000
for i in range(m2):
    if ytest[i]<5:
        pt_xtest.append(xtest[i] /255)
        pt_ytest.append(ytest[i])
    else:
        tl_xtest.append(xtest[i] / 255)
        tl_ytest.append(ytest[i])


In [5]:
from keras.utils import np_utils

In [6]:
# converting data into numpy arrays.
pt_xtrain = np.asarray(pt_xtrain).reshape(-1, 28, 28, 1)
pt_xtest = np.asarray(pt_xtest).reshape(-1, 28, 28, 1)
pt_ytrain = np_utils.to_categorical(np.asarray(pt_ytrain))
pt_ytest = np_utils.to_categorical(np.asarray(pt_ytest))

tl_xtrain = np.asarray(tl_xtrain).reshape(-1, 28, 28, 1)
tl_xtest = np.asarray(tl_xtest).reshape(-1, 28, 28, 1)
tl_ytrain = np_utils.to_categorical(np.asarray(tl_ytrain))
tl_ytest = np_utils.to_categorical(np.asarray(tl_ytest))

In [7]:
print(pt_xtrain.shape, pt_xtest.shape)
print(pt_ytrain.shape, pt_ytest.shape)

print(tl_xtrain.shape, tl_xtest.shape)
print(tl_ytrain.shape, tl_ytest.shape)

(10225, 28, 28, 1) (5139, 28, 28, 1)
(10225, 5) (5139, 5)
(9775, 28, 28, 1) (4861, 28, 28, 1)
(9775, 10) (4861, 10)


In [8]:
from keras.models import Sequential
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPool2D 

In [9]:
model = Sequential()

model.add(Conv2D(32, 5, input_shape=(28, 28, 1), activation='relu'))
model.add(Conv2D(16, 5, activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(8, 3, activation='relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 16)        12816     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 8)           1160      
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               6

In [10]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [11]:
model.fit(pt_xtrain, pt_ytrain,
         batch_size=100,
         epochs =10,
         shuffle=True,
         verbose=2,
         validation_data=(pt_xtest, pt_ytest))

Epoch 1/10
103/103 - 14s - loss: 0.3090 - accuracy: 0.8967 - val_loss: 0.0557 - val_accuracy: 0.9835
Epoch 2/10
103/103 - 14s - loss: 0.0893 - accuracy: 0.9733 - val_loss: 0.0326 - val_accuracy: 0.9905
Epoch 3/10
103/103 - 14s - loss: 0.0557 - accuracy: 0.9817 - val_loss: 0.0248 - val_accuracy: 0.9909
Epoch 4/10
103/103 - 14s - loss: 0.0475 - accuracy: 0.9843 - val_loss: 0.0213 - val_accuracy: 0.9938
Epoch 5/10
103/103 - 14s - loss: 0.0330 - accuracy: 0.9891 - val_loss: 0.0140 - val_accuracy: 0.9955
Epoch 6/10
103/103 - 14s - loss: 0.0298 - accuracy: 0.9907 - val_loss: 0.0155 - val_accuracy: 0.9949
Epoch 7/10
103/103 - 14s - loss: 0.0232 - accuracy: 0.9918 - val_loss: 0.0170 - val_accuracy: 0.9946
Epoch 8/10
103/103 - 15s - loss: 0.0220 - accuracy: 0.9927 - val_loss: 0.0174 - val_accuracy: 0.9951
Epoch 9/10
103/103 - 14s - loss: 0.0192 - accuracy: 0.9937 - val_loss: 0.0108 - val_accuracy: 0.9959
Epoch 10/10
103/103 - 14s - loss: 0.0167 - accuracy: 0.9946 - val_loss: 0.0088 - val_accura

In [12]:
model.layers

In [14]:
# freezing the conv layers of the pretrained model

for layer in model.layers[:6]:
    layer.trainable=False

for layer in model.layers:
    print(layer.trainable)

False
False
False
False
False
False
True
True
True
True


In [15]:
# model for transfer learning
tl_model = Sequential(model.layers[:6])
tl_model.add(Dense(128))
tl_model.add(Activation('relu'))
tl_model.add(Dense(10))
tl_model.add(Activation('softmax'))

tl_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 16)        12816     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 8)           1160      
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)              

In [16]:
tl_model.compile(loss='categorical_crossentropy',
                optimizer = 'adam',
                metrics=['accuracy'])

In [18]:
tl_model.fit(tl_xtrain, tl_ytrain,
            batch_size=100,
            epochs=10,
            shuffle=True,
            verbose=2,
            validation_data=(tl_xtest, tl_ytest))

Epoch 1/10
98/98 - 2s - loss: 0.3024 - accuracy: 0.9005 - val_loss: 0.0779 - val_accuracy: 0.9749
Epoch 2/10
98/98 - 2s - loss: 0.0964 - accuracy: 0.9692 - val_loss: 0.0493 - val_accuracy: 0.9858
Epoch 3/10
98/98 - 2s - loss: 0.0762 - accuracy: 0.9766 - val_loss: 0.0430 - val_accuracy: 0.9854
Epoch 4/10
98/98 - 2s - loss: 0.0682 - accuracy: 0.9780 - val_loss: 0.0372 - val_accuracy: 0.9877
Epoch 5/10
98/98 - 2s - loss: 0.0557 - accuracy: 0.9812 - val_loss: 0.0369 - val_accuracy: 0.9877
Epoch 6/10
98/98 - 2s - loss: 0.0534 - accuracy: 0.9818 - val_loss: 0.0350 - val_accuracy: 0.9879
Epoch 7/10
98/98 - 2s - loss: 0.0471 - accuracy: 0.9846 - val_loss: 0.0347 - val_accuracy: 0.9891
Epoch 8/10
98/98 - 3s - loss: 0.0474 - accuracy: 0.9840 - val_loss: 0.0283 - val_accuracy: 0.9887
Epoch 9/10
98/98 - 3s - loss: 0.0420 - accuracy: 0.9860 - val_loss: 0.0323 - val_accuracy: 0.9885
Epoch 10/10
98/98 - 3s - loss: 0.0420 - accuracy: 0.9871 - val_loss: 0.0314 - val_accuracy: 0.9901
